In [48]:
# Import Packages

import pandas as pd
import numpy as np
import os
import pickle

In [49]:
teams = pd.read_csv('playoff_teams.csv')
all_pitching_stats = pd.read_csv('baseball_data/pitching.csv')

In [61]:
pitching_features = ['id', 'p_ipouts', 'p_h', 'p_d', 'p_t', 'p_hr', 'p_r', 'p_er', 'p_w', 'p_k', 'p_hbp', 'p_wp', 'p_bk', 'p_sh', 'p_sf', 'wp', 'lp', 'save', 'p_cg']
pitching_features_final = ['p_d', 'p_t', 'p_hr', 'p_r', 'p_k', 'p_hbp', 'p_wp', 'p_bk', 'psh', 'p_sf', 'wp', 'lp', 'save', 'p_cg', 'whip', 'era']

In [70]:
max_players = 25
num_features = len(pitching_features_final)

all_seasons = np.empty([0, max_players, num_features])

for index, team in teams.iterrows():
    print(f"Index: {index}, Year: {team['Year']} Team: {team['Team']}, Level of Success: {team['Level of Success']}")
    temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
    temp_stats = temp_stats.groupby(by='id').sum()

    # (Total number of walks + Total number of hits) / Total number of innings pitched
    temp_stats['whip'] = (temp_stats['p_w'] + temp_stats['p_h']) / temp_stats['p_ipouts']
    # (Earned Runs / Innings Pitched) x 9
    temp_stats['era'] = (temp_stats['p_er'] / temp_stats['p_ipouts']) * 9

    temp_stats = temp_stats.sort_values(by=['p_ipouts', 'era'], ascending=[False, True])
    temp_stats = temp_stats.drop(columns=['p_ipouts', 'p_er', 'p_h', 'p_w'])
    print('Max WHIP:', np.max(temp_stats['whip']))
    print('Min WHIP:', np.min(temp_stats['whip']))
    print('Max ERA:', np.max(temp_stats['era']))
    print('Min ERA:', np.min(temp_stats['era']))
    # print(temp_stats.columns)
    temp_stats = temp_stats.to_numpy()
    print(temp_stats.shape)

    padding = max_players - temp_stats.shape[0]
    if padding > 0:
        temp_stats = np.pad(temp_stats, ((0, padding), (0, 0)), mode='constant')
    elif padding < 0:
        temp_stats = temp_stats[:max_players,]
    temp_stats = np.reshape(temp_stats, (1, max_players, num_features))

    print(temp_stats.shape)
    print('Max WHIP:', np.max(temp_stats[0,:,-2]))
    print('Min WHIP:', np.min(temp_stats[0,:,-2]))
    print('Max ERA:', np.max(temp_stats[0,:,-1]))
    print('Min ERA:', np.min(temp_stats[0,:,-1]))
    all_seasons = np.vstack((all_seasons, temp_stats))

# with open('all_pitching_data.pkl', 'wb') as file:
#     pickle.dump(all_seasons, file)

# print("Data saved to 'all_pitching_data.pkl'")

Index: 0, Year: 1995 Team: BOS, Level of Success: DS


/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]


Max WHIP: 2.0
Min WHIP: 0.3626373626373626
Max ERA: 12.0
Min ERA: 0.0
(26, 16)
(1, 25, 16)
Max WHIP: 1.4
Min WHIP: 0.3626373626373626
Max ERA: 6.3
Min ERA: 0.0
Index: 1, Year: 1995 Team: CLE, Level of Success: WS


/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]


Max WHIP: 0.875
Min WHIP: 0.34375
Max ERA: 4.5
Min ERA: 0.375
(20, 16)
(1, 25, 16)
Max WHIP: 0.875
Min WHIP: 0.0
Max ERA: 4.5
Min ERA: 0.0
Index: 2, Year: 1995 Team: SEA, Level of Success: LS


...

/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]


Max WHIP: 2.0
Min WHIP: 0.25
Max ERA: 12.0
Min ERA: 0.5625
(26, 16)
(1, 25, 16)
Max WHIP: 2.0
Min WHIP: 0.25
Max ERA: 9.0
Min ERA: 0.5625
Index: 264, Year: 2023 Team: MIA, Level of Success: WC


/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]


Max WHIP: 3.0
Min WHIP: 0.32905982905982906
Max ERA: 13.5
Min ERA: 0.0
(32, 16)
(1, 25, 16)
Max WHIP: 0.8285714285714286
Min WHIP: 0.32905982905982906
Max ERA: 3.789473684210526
Min ERA: 0.7692307692307693
Index: 265, Year: 2023 Team: ARI, Level of Success: WS
Max WHIP: 1.1666666666666667
Min WHIP: 0.0
Max ERA: 6.0
Min ERA: 0.0
(34, 16)
(1, 25, 16)
Max WHIP: 0.8
Min WHIP: 0.1794871794871795
Max ERA: 4.2
Min ERA: 0.0


/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
/var/folders/fh/fzyqqz2s1vzcm3v06kz1h29w0000gp/T/ipykernel_2244/85417958.py:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  temp_stats = all_pitching_stats[all_pitching_stats['team'] == team['Team']][all_pitching_stats['date'].astype(str).str.startswith(str(team['Year']))][all_pitching_stats['gametype'] == 'regular'][pitching_features]
